In [ ]:
!pip install --q crewai
!pip install --q langchain_groq
!pip install --q langchain_chatOpenAI
!pip install --q 'crewai[tools]'
!pip install --q dotenv
!pip install --q textwrap3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.1/78.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.6/327.6 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.5/339.5 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
import os

from crewai_tools.tools.scrape_website_tool.scrape_website_tool import ScrapeWebsiteTool
from dotenv import load_dotenv
from datetime import datetime
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from textwrap import dedent

In [ ]:
# Carregar as variáveis de ambiente do arquivo .env
load_dotenv()

# Configuração das chaves de API (substitua com suas chaves reais)
openai_api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

gpt3_llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=openai_api_key)
gpt4o_llm = ChatOpenAI(model="gpt-4o", api_key=openai_api_key)
llama3_70b = ChatGroq(model="llama3-70b-8192", api_key=groq_api_key)


#FERRAMENTAS QUE A AI VAI USAR, NO CASO AQUI SÃO OS SITES
ICL = ScrapeWebsiteTool(
    website_url="https://iclnoticias.com.br/"
)

OPovo = ScrapeWebsiteTool(
    website_url="https://www.opovo.com.br/"
)

RAutomacao = ScrapeWebsiteTool(

    website_url="https://revista-automacao.com/"
)
DN = ScrapeWebsiteTool(
    website_url="https://diariodonordeste.verdesmares.com.br/"
)

In [ ]:
#DESCRIÇÃO DOS AGENTES COM SEUS PROMPTS
planejador = Agent(
    role="Planejador de postagem",
    goal="Planejar conteúdo envolvente para instagram sobre {topic}",
    backstory="Você está trabalhando no planejamento de {n} posts para o instagram "
              "sobre o tema: {topic}. "
              "Você coleta informações que ajudam o "
              "público se informar sobre {topic}. "
              "Seu trabalho é a base para que "
              "o Pesquisador de Conteúdo procure na web sobre {topic}.",
    verbose=True,
    llm=gpt3_llm,
    allow_delegation=False

)

pesquisador = Agent(
    role='Pesquisador',
    goal='Pesquisar tendências para postagens sobre {topic} na área '
         'de tecnologia com base no planejamento do Planejador. '
         'Seu trabalho é a base para que '
         'o escritor possa escrever {n} posts sobre {topic}',
    verbose=True,
    backstory="Você é um pesquisador experiente, sempre em busca das últimas tendências e informações relevantes sobre {topic}.",
    llm=gpt3_llm,
    tools=[ DN, OPovo],
    allow_delegation=False
)



escritor = Agent(
    role='Escritor',
    goal='Escrever {n} postagens cativantes em português do Brasil para o Instagram sobre {topic} com no mínimo 250 palavras e no máximo 350 palavras.'
         'Seu trabalho é a base para que o fotografo possa escrever prompts de imagens para os {n} posts',
    backstory="Você é um escritor criativo, capaz de transformar informações em conteúdo atraente para postagens no Instagram.",
    llm=gpt4o_llm,
    verbose=True,
    allow_delegation=False
)


fotografo = Agent(
    role='Fotógrafo',
    goal='Escrever prompts de imagens para as {n} postagens para gerar imagens cativantes para o Instagram sobre {topic}.',
    backstory=dedent("""Você é um fotógrafo criativo,
                        capaz de transformar informações em imagens e escrever prompts
                        de imagens atraentes para postagens no Instagram."""),
    verbose=True,
    llm=llama3_70b,
    max_rpm=1,
    allow_delegation=False
)



gerente = Agent(
    role='Gerente de postagens',
    goal=dedent("""Supervisione o trabalho de uma equipe de postagens no Instagram. Você é bem crítico em relação
            ao que vai ser postado no Instagram da empresa de notícias na área da tecnologia.
            Você delegará tarefas à sua equipe e fará perguntas esclarecedoras
            para revisar e aprovar as {n} posts sobre {topic} que foram solicitadas pela direção da empresa."""),
    verbose=True,
    backstory=dedent("""Você é um gerente experiente, sempre em busca das últimas tendências e informações relevantes.
                 Você está trabalhando com uma nova demanda e faz com que sua equipe realize o trabalho da
                 melhor forma possível."""),
    llm=gpt3_llm,
)


#DESCRIÇÃO DAS TAREFAS
# TASKS
plano_task = Task(
    description=(
        "1. Priorize as últimas tendências, principais 'players', "
            "e notícias relevantes sobre {topic}.\n"
        "2. Identifique o público-alvo, considerando "
            "seus interesses e pontos de dor.\n"
        "3. Desenvolva um plano de conteúdo detalhado, incluindo "
            "uma introdução, pontos principais e um chamado à ação.\n"
        "4. Inclua palavras-chave de SEO e dados ou fontes relevantes."
    ),
    expected_output="Um documento de plano de conteúdo para {n} posts sobre {topic} "
        "com um esboço, análise do público, "
        "palavras-chave de SEO e recursos.",
    agent=planejador,
    verbose=2
)


pesquisa_task = Task(
    description="Pesquise as últimas tendências sobre {topic}.",
    expected_output="Um relatório detalhado sobre as tendências mais recentes sobre {topic} na área de tecnologia.",
    agent=pesquisador,
    verbose=2
)


escrita_task = Task(
    description=dedent("""Escreva {n} postagens envolventes em português do Brasil
                        com base nas tendências pesquisadas sobre {topic} com no mínimo 250 palavras e no máximo 350 palavras cada.
                        Cada post deve ser formatado como:
                        \n\nPOST:\ntexto do post em português do brasil
                        \n\nPROMPT:\nPrompt da imagem desse post\n\n"""),
    expected_output="{n} postagens de Instagram bem escritas, atraentes e em português do Brasil, formatadas conforme especificado para o tópico {topic}.",
    agent=escritor,
    verbose=2
)


criacao_imagem_task = Task(
    description="Crie {n} prompts em inglês, para criar uma imagem atraente para acompanhar a postagem no Instagram sobre {topic}.",
    expected_output="{n} prompts de alta qualidade adequados para o Instagram based in {topic}.",
    agent=fotografo,
    verbose=2
)


revisao_task = Task(
    description=dedent("""Revise as {n} escritas e prompts de imagens
                          para as {n} postagens do cliente e garanta
                          que as informações de cada postagem estejam organizadas, sem erros e cativantes
                          em português do Brasil sobre {topic}.
                          Certifique-se de que cada post está formatado como:
                          \n\nPOST:\ntexto do post em português do brasil
                          \n\nPROMPT:\nPrompt da imagem desse post\n\n"""),
    expected_output="{n} textos e prompts de imagens organizados por post, revisados e prontos para serem publicados em português do Brasil, formatados conforme especificado.",
    agent=gerente,
    verbose=2
)

In [ ]:
crew = Crew(
    agents=[planejador, pesquisador, escritor, fotografo, gerente],
    tasks=[plano_task, pesquisa_task, escrita_task, criacao_imagem_task, revisao_task],
    process=Process.sequential,
    verbose=2,
    memory=True
)


In [ ]:
result = crew.kickoff(inputs={
      'topic':'Saude e farmacia',
      'n':1})


 [2024-07-11 00:43:38][DEBUG]: == Working Agent: Planejador de postagem
 [2024-07-11 00:43:38][INFO]: == Starting Task: 1. Priorize as últimas tendências, principais 'players', e notícias relevantes sobre Saude e farmacia.
2. Identifique o público-alvo, considerando seus interesses e pontos de dor.
3. Desenvolva um plano de conteúdo detalhado, incluindo uma introdução, pontos principais e um chamado à ação.
4. Inclua palavras-chave de SEO e dados ou fontes relevantes.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Plano de Conteúdo para Post sobre Saúde e Farmácia no Instagram:

Introdução:
Neste post, vamos abordar as últimas tendências e notícias relevantes sobre saúde e farmácia, visando informar e educar nosso público-alvo sobre questões importantes relacionadas a esses temas. Além disso, iremos destacar os principais 'players' do setor e oferecer um chamado à ação para incentivar a interação e engajamento dos seguidores.

Público-Alvo:
Nos

In [ ]:
current_date = datetime.now().strftime("%Y-%m-%d")
filename = f"posts-{current_date}.txt"              #"post-2024-06-11.txt"
with open(filename, 'w', encoding='utf-8') as file:
    file.write(result)